<a href="https://colab.research.google.com/github/wksmirnowa/machinelearning_homeworks/blob/master/HW05_ensemble_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ансамбли



## Импорты и важные функции

In [69]:
%matplotlib inline

import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions

import numpy as np
import pandas as pd
import gensim
from string import punctuation
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression, LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier,  BaggingClassifier, BaggingRegressor, RandomTreesEmbedding, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from mlxtend.classifier import StackingClassifier
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')
lem = WordNetLemmatizer()

punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('english'))

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]
    words = [word for word in words if word]

    return words

def normalize_en(text):

  words = []
  tokenized_text = tokenize(text)
  for word, tag in pos_tag(tokenized_text):
    lemtag = tag[0].lower()
    lemtag = lemtag if lemtag in ['a', 'r', 'n', 'v'] else None

    if not lemtag:
      lemma = word
      words.append(lemma)
    else:
      lemma = lem.lemmatize(word, lemtag)
      words.append(lemma)

  return " ".join(words)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!wget https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv

--2020-05-24 14:17:52--  https://raw.githubusercontent.com/TatianaShavrina/hse_ml_m1/master/ensembles/complaints.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4148456 (4.0M) [text/plain]
Saving to: ‘complaints.csv’

complaints.csv      100%[===================>]   3.96M  --.-KB/s    in 0.1s    

2020-05-24 14:17:52 (29.8 MB/s) - ‘complaints.csv’ saved [4148456/4148456]



In [0]:
def show_metrics(y_test, predictions):
  print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
  print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
  print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
  print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))

## Работа с данными

In [4]:
data = pd.read_csv('complaints.csv', sep='\t')
data.head()

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv..."
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...


Предобработаем тексты с помощью nltk. В данных много несловарных форм, но попадаются и обычные словарные слова, с ними лемматизация должна сработать и тем самым немного повысить метрики

In [0]:
norm_texts = []
for line in data.cleaned_text:
  norm_texts.append(normalize_en(line))

In [0]:
data['norm_text'] = norm_texts

In [7]:
data.head(10)

,COMPLAINT_ID,DATE,PRODUCT_ID,ISSUE_ID,cleaned_text,norm_text
0,3178905,03/13/2019,44,318,go year . contact advis never took loan . advi...,go year contact advis never take loan advis se...
1,3175952,03/12/2019,44,349,"mail valid debt xx/xx/19 valid receiv , receiv...",mail valid debt xx/xx/19 valid receiv receiv c...
2,3174747,03/09/2019,44,16,xx/xx/xxxx appli receiv onlin loan bluechip fi...,xx/xx/xxxx appli receiv onlin loan bluechip fi...
3,3173291,03/08/2019,44,16,xx/xx/xxxx appli receiv onlin loan . loan amou...,xx/xx/xxxx appli receiv onlin loan loan amount...
4,3172221,03/07/2019,44,48,told husband left bill . debt would pay within...,told husband leave bill debt would pay within ...
5,3172723,03/07/2019,44,310,"payday loan , becam abl pay . everyon famili c...",payday loan becam abl pay everyon famili conta...
6,3171497,03/06/2019,44,320,"within past week , contact oracl financi group...",within past week contact oracl financi group l...
7,3171674,03/06/2019,44,103,call work said courthous owe money . got phone...,call work say courthous owe money get phone wo...
8,3170784,03/06/2019,44,318,today xx/xx/xxxx cst contact ks . receiv appro...,today xx/xx/xxxx cst contact k receiv approxim...
9,3171479,03/06/2019,44,19,cashcal attempt collect fraudul debt . ident s...,cashcal attempt collect fraudul debt ident ste...


## Задание 1. Voting Classifier

Задание: попробуйте поднять качество классификации до 70% с помощью известных вам методов препроцессинга и Voting

### Способ 1.

In [0]:
y = data["PRODUCT_ID"]
X = data["norm_text"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) #test_size=0.2

In [0]:
clf1 = LogisticRegression(penalty='l2', C=0.9, multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
clf2 = RandomForestClassifier(n_estimators=200, criterion='gini', max_depth=20, min_samples_leaf=5, max_features=300, random_state=42, n_jobs=-1) #class_weight='balanced'
clf3 = GaussianNB(var_smoothing=1e-90)

eclf = VotingClassifier(estimators=[
        ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard', n_jobs=10)

voting = Pipeline([
    ('vect', CountVectorizer(min_df=2, analyzer='word', max_features=500, ngram_range=(1,3), stop_words='english')), #ngram_range=(1,2), max_df=300, 
    ('tfidf', TfidfTransformer(norm='l2', sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('clf', eclf),
    ])
voting = voting.fit(X_train, y_train)
predictions = voting.predict(X_test)
show_metrics(y_test, predictions)

Precision:   0.70
Recall:   0.65
F1-measure:   0.65
Accuracy:   0.65


### Способ 2.

In [0]:
model = gensim.models.Word2Vec(X_train, size=100)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = 100

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in word_tokenize(words) if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [0]:
clf = Pipeline([
    ('feats', FeatureUnion([
        ('tfidf', TfidfVectorizer(min_df=2, analyzer='word', max_features=500, ngram_range=(1,3), stop_words='english')),
        ("word2vec vectorizer", MeanEmbeddingVectorizer(model))])),
    ('clf', ExtraTreesClassifier(n_estimators=200))
    ])

clf = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
show_metrics(y_test, predictions)

Precision:   0.71
Recall:   0.70
F1-measure:   0.70
Accuracy:   0.70


## Задание 2.

Основываясь на своих знаниях об ансамблировании различных моделей, на данных из complaints.csv достигните наилучшего результата

### Bagging

In [0]:
np.random.seed(0)

In [0]:
cv = CountVectorizer(min_df=2, analyzer='word', max_features=500, ngram_range=(1,3), stop_words='english')
y = data["PRODUCT_ID"]
X = cv.fit_transform(data["norm_text"])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
clf1 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=5, min_samples_split=3, min_samples_leaf=2, max_features=100, random_state=42)
clf2 = SVC(C=5.0, kernel='poly', degree=5, gamma='auto', coef0=1.0, max_iter=1000, random_state=42)
clf3 = MultinomialNB(alpha=0.5)    
clf4 = LogisticRegression(penalty='l2', C=0.9, multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)

bagging1 = BaggingClassifier(base_estimator=clf1, n_estimators=200, max_samples=0.9, max_features=0.9, n_jobs=5)
bagging2 = BaggingClassifier(base_estimator=clf2, n_estimators=200, max_samples=0.9, max_features=0.9, n_jobs=5)
bagging3 = BaggingClassifier(base_estimator=clf3, n_estimators=200, max_samples=0.9, max_features=0.9, n_jobs=5)
bagging4 = BaggingClassifier(base_estimator=clf4, n_estimators=200, max_samples=0.9, max_features=0.9, n_jobs=5)

In [0]:
kfold = KFold(n_splits=3, random_state=42)

In [41]:
%%time
clf_list = [bagging1, bagging2, bagging3, bagging4]
for clf in clf_list:        
    scores = cross_val_score(clf, X, y, cv=kfold, scoring='accuracy')
    print(f'{clf}')
    print ("Accuracy: %.2f (+/- %.2f)" %(scores.mean(), scores.std()))
        
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    show_metrics(y_test, predictions)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=5,
                                                        max_features=100,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=2,
                                                        min_samples_split=3,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=42,
   

In [26]:
clf_rf = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_leaf=5,
                             class_weight='balanced', random_state=42)


scores = cross_val_score(clf_rf, X, y, cv=kfold, scoring='accuracy')
print(f'{clf_rf}')
print ("Accuracy: %.2f (+/- %.2f)" %(scores.mean(), scores.std()))
        
clf_rf.fit(X_train, y_train)
predictions = clf_rf.predict(X_test)
show_metrics(y_test, predictions)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Accuracy: 0.49 (+/- 0.22)
Precision:   0.68
Recall:   0.67
F1-measure:   0.67
Accuracy:   0.67


In [33]:
clf_et = ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=20, min_samples_leaf=5, max_features=300, bootstrap=True,
                             class_weight='balanced', random_state=0, max_samples=300)


scores = cross_val_score(clf_et, X, y, cv=kfold, scoring='accuracy')
print(f'{clf_et}')
print ("Accuracy: %.2f (+/- %.2f)" %(scores.mean(), scores.std()))
        
clf_et.fit(X_train, y_train)
predictions = clf_et.predict(X_test)
show_metrics(y_test, predictions)

ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                     criterion='gini', max_depth=20, max_features=300,
                     max_leaf_nodes=None, max_samples=300,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=5, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=200,
                     n_jobs=None, oob_score=False, random_state=0, verbose=0,
                     warm_start=False)
Accuracy: 0.36 (+/- 0.24)
Precision:   0.64
Recall:   0.62
F1-measure:   0.62
Accuracy:   0.62


### Boosting

In [0]:
cv = CountVectorizer(min_df=2, analyzer='word', max_features=500, ngram_range=(1,3), stop_words='english')
y = data["PRODUCT_ID"]
X = cv.fit_transform(data["norm_text"])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
clf1 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=5, min_samples_split=3, min_samples_leaf=2, max_features=100, random_state=42)
clf2 = LogisticRegression(penalty='l2', C=0.9, multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
clf3 = MultinomialNB(alpha=0.5)  

In [0]:
kfold = KFold(n_splits=3, random_state=42)

In [62]:
%%time
num_est = [1, 2, 3, 5]
clf_list = [clf1, clf2, clf3]
labels = ['AdaBoost (n_est=10)', 'AdaBoost (n_est=50)', 'AdaBoost (n_est=100)', 'AdaBoost (n_est=200)']
for n_est in num_est:
  for clf in clf_list:
    print(f'{n_est}, {clf}')
    boosting = AdaBoostClassifier(base_estimator=clf, n_estimators=n_est)  
    boosting.fit(X, y)
    predictions = clf_et.predict(X)
    show_metrics(y, predictions)
    scores = cross_val_score(boosting, X, y, cv=kfold, scoring='accuracy')
    print ("Accuracy: %.2f (+/- %.2f)" %(scores.mean(), scores.std()))



1, DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=5, max_features=100, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='random')
Precision:   0.66
Recall:   0.64
F1-measure:   0.64
Accuracy:   0.64
Accuracy: 0.10 (+/- 0.10)
1, LogisticRegression(C=0.9, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Precision:   0.66
Recall:   0.64
F1-measure:   0.64
Accuracy:   0.64
Accuracy: 0.12 (+/- 0.11)
1, MultinomialNB(alpha=0.5, class_prior=None, 

In [0]:
import xgboost as xgb

In [65]:
clf = xgb.XGBClassifier(objective='multi:softmax')
clf.fit(X_train,  y_train)
predictions = clf.predict(X_test)
show_metrics(y_test, predictions)
scores = cross_val_score(sclf, X, y, cv=kfold, scoring='accuracy')
print ("Accuracy: %.2f (+/- %.2f)" %(scores.mean(), scores.std()))

Precision:   0.65
Recall:   0.64
F1-measure:   0.64
Accuracy:   0.64
Accuracy: 0.23 (+/- 0.17)


### Stacking

In [0]:
cv = CountVectorizer(min_df=2, analyzer='word', max_features=500, ngram_range=(1,3), stop_words='english')
y = data["PRODUCT_ID"]
X = cv.fit_transform(data["norm_text"])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [0]:
clf1 = DecisionTreeClassifier(criterion='entropy', splitter='random', max_depth=5, min_samples_split=3, min_samples_leaf=2, max_features=100, random_state=42)
clf2 = SVC(C=5.0, kernel='poly', degree=5, gamma='auto', coef0=1.0, max_iter=1000, random_state=42)
clf3 = MultinomialNB(alpha=0.5) 
clf4 = LogisticRegression(penalty='l2', C=0.9, multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], 
                          meta_classifier=clf4)

In [67]:
sclf.fit(X, y)
predictions = sclf.predict(X)
show_metrics(y, predictions)
scores = cross_val_score(sclf, X, y, cv=kfold, scoring='accuracy')
print ("Accuracy: %.2f (+/- %.2f)" %(scores.mean(), scores.std()))

Precision:   0.59
Recall:   0.59
F1-measure:   0.54
Accuracy:   0.60
Accuracy: 0.23 (+/- 0.17)
